In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
base_url = "https://comic.naver.com/webtoon/weekday.nhn"

os.chdir(r"E:\Programming\python\NLP\Crawling\chromedriver_win32")

In [3]:
# driver 실행
def drive(url):
    # driver 객체 불러오기
    driver = webdriver.Chrome("./chromedriver")
    # 3초 후에 작동하도록 설정
    driver.implicitly_wait(3)
    # url에 접속
    driver.get(url)
    # 현재 나타난 창의 page_sourc(html) 가져오기
    html = driver.page_source
    # html 파싱을 위해 beautifulsoup에 넘겨주기
    soup = BeautifulSoup(html, 'html.parser')

    return driver, soup

In [4]:
# 웹툰 기본페이지에서 데이터 가져오기
driver, soup = drive(base_url)
driver.close()

# 가져온 데이터 파싱, id, 요일, 이름
title = soup.select('.title')
t_IDs = list(map(lambda x: x.get('href').split('titleId=')[1].split('&')[0], title))
t_weekdays = list(map(lambda x: x.get('href').split('weekday=')[1], title))
t_names = list(map(lambda x: x.text, title))

# 데이터 확인하기
print('t_IDs: ', len(t_IDs), t_IDs[0])
print('t_weekdays: ', len(t_weekdays), t_weekdays[0])
print('t_names: ', len(t_names), t_names[0])

t_IDs:  338 183559
t_weekdays:  338 mon
t_names:  338 신의 탑


In [19]:
# 웹툰 이름으로 id와 weekday 반환
def find_id_weekday(name, t_names, t_IDs, t_weekdays, start_idx=0):
    try:
        idx = t_names.index(name)
        print(t_IDs[idx])
    except:
        print('찾는 웹툰이 없습니다.')
        return
    return t_IDs[idx], t_weekdays[idx]

In [20]:
# 회차 수 세기
def episode_cnt(ID, weekday):
    url = base_url.split('weekday')[0]+'list.nhn?titleId={0}&weekday={1}'.format(ID, weekday)
    driver, soup = drive(url)
    driver.close()
    res = soup.select('.title')[0].select('a')[0].get('href').split('no=')[1].split('&')[0]

    return res

res = episode_cnt(183559, 'mon')
print(res)

481


In [28]:
def comment_crawler(name, start_idx=0):
    id_num, weekday = find_id_weekday(name, t_names, t_IDs, t_weekdays, start_idx=start_idx)
    cnt = int(episode_cnt(id_num, weekday))

    print(cnt)

    comments = []
    # 진행 상태 표시를 위한 숫자. 0보다 작아야 0%가 표시 된다
    proceed = -1 

    # driver만 먼저 열어 놓는다. for문을 돌면서 url만 바꿔줄 것이다
    driver, _ = drive(base_url)
    print("진행 중...")
    for i in range(1, cnt+1):
        percentage = int((i/cnt)*100)
        # 진행 상태 표시
        if percentage%10==0 and percentage > proceed:
            proceed = percentage
            print("{0}% 완료".format(proceed))
        url = "https://comic.naver.com/comment/comment.nhn?titleId={0}&no={1}#".format(id_num, str(i))
        print(url)
        time.sleep(1.5)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        comments += list(map(lambda x: x.text, soup.select('.u_cbox_contents')))

    driver.close()
    print("complete")

    return comments

In [29]:
comments = comment_crawler('오로지 너를 이기고 싶어')

730657
46
진행 중...
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=1#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=2#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=3#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=4#
10% 완료
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=5#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=6#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=7#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=8#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=9#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=10#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=11#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=12#
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=13#
30% 완료
https://comic.naver.com/comment/comment.nhn?titleId=730657&no=14#
https://comic.naver.com/comment/comment.nhn?titleId=7

In [30]:
print(comments)

너무 부럽다내 인생에 여자는 누나들이랑 엄마밖에 없는데', '이야 둘이 아주 달달한게 저기있는 아이스크림 다 덤벼도 못이기겠네 ㅋㅋㅋ', '오로지 막컷 너무 이쁘다', '이기고 너 뭐야 이기곤데요ㅋㅋㅋㅋㅋㅋㅋ', '다음에 떡볶이 사줄거잖아 = 자연스러운 데이트 신청', '민트초코 드립을 이해하신 분들만 연애1단계를 성공하실수 있습니다', '배라 먹을빠엔 뜨근하고 든든한 국밥말어먹고말지', '뺀찌맞은 민트초코입니다. 별점 1점 드리겠습니다.', '기승전 국밥충 365일 하루3끼 국밥만 먹여보고싶네', '난 뽀뽀하려고 민트초코 넣은 줄 알았는데', '달달해 넘 좋아', '와씨 배뎃중에서 누나들을 여자로 쳐 주다니;; 내 동생아 좀 본받아라 난 형이아니다;;;;;', '라면 벌써 다 불었으니까 그만 불어 ㅋㅋㅋㅋㅋㅋㅋ', '로지 앞머리넘긴거 완전 예뻐 그리고 별빛천사 뭐야 대박 ㅜㅜㅜㅜ', '이기고 프사 뭔데 너무 귀엽다ㅋㅋㅋㅋㅋㄷㅋㅋㅋ', '와 오로지 너무 예쁘다', '원래 민초 싫어했는데 이번화 보고 민초가 부러워졌습니다;;', '속보입니다.대한민국 솔로들 웹툰보고 단체사망', '수능 4일 남았는데 이거 보면서 히히덕 거리는 내 인생이 레전드다', '오로지 넘 예뻐', '만화책 달빛천사이네요ㅠㅜ 추억ㅠ 이번에 리메이크 앨범도 나왔고 재방영도 한다는데 진짜 추억돋네요..ㅠ', '저 소원 사귀자는 소원 아님??', '이기고 선씻을때 봉숭아 물들인 손가락 빼고 씻는거 대박 ㅜㅡ', '너네 세명 빼고 교직원 포함 학교에 모든 사람들이 아는데 그걸 질문이라고 하나', '나 사실 너한테 관심있어 이기고', '여지껏 봐왔던 주인공 특성상 백퍼 확답 못하고 "로지는 그냥 내 친구야!!" 라고 나오는 고구마 전개다 =ㅅ=', '작가양반....빨리 다음편을 안주면 바지에 똥을 싸겠네...', '아니라하면 안본다 아니 그래도 볼거같긴한데 아니라 할 바엔 그냥 쟤 벽치기 해버려', '나도 순정이 있다 파랑아', '이기고: 당연하지 이제 가줄래?', '이거 괜히 어물쩍 거리다가 파랭이